In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob

"""Change to the data folder"""
new_path = "./new_train/new_train"
#new_path = r"C:\Users\mnisyu\Desktop\CSE151B competition\cse151b-spring\\"
#new_path = ".."

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [10]:
import pickle

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
train_data  = ArgoverseDataset(data_path="../input/kaggle-competitions-download-c-cse151bspring/new_train/new_train/")
test_data  = ArgoverseDataset(data_path="../input/kaggle-competitions-download-c-cse151bspring/new_val_in/new_val_in/")

### Create a loader to enable batch processing

# By Lehan Li

In [104]:
import itertools
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

batch_sz = 4

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [np.dstack([scene['p_in'][:,:,0].reshape(60*19),scene['p_in'][:,:,1].reshape(60*19),scene['v_in'][:,:,0].reshape(60*19),scene['v_in'][:,:,1].reshape(60*19)]).reshape(60*19,4) for scene in batch]
    
    indexs = [np.where(scene['agent_id'] == scene['track_id'][:,0].reshape(60))[0][0] for scene in batch]
    #p_in= [scene['p_in'][index] for (scene,index) in zip(batch, indexs)]
    #v_in = [scene['v_in'][index] for (scene,index) in zip(batch, indexs)]
    #inp = [np.concatenate((p[:,0],p[:,1],v[:,0],v[:,1])).reshape(76,1) for(p,v) in zip(p_in,v_in)]
    
    p_out = [scene['p_out'][index] for (scene,index) in zip(batch, indexs)]
    v_out = [scene['v_out'][index] for (scene,index) in zip(batch, indexs)]
    out = [np.concatenate((p[:,0],p[:,1],v[:,0],v[:,1])) for(p,v) in zip(p_out,v_out)]
    
    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    return [inp, out]

train_loader = DataLoader(train_data,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0, drop_last=True)

In [59]:
_, (example_datas, labels) = next(enumerate(train_loader))
example_datas.size()
#labels.size()

torch.Size([4, 1140, 4])

In [62]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = torch.nn.LSTM(4, 1, 4, dropout=0.2)
        self.conv1 = torch.nn.Conv1d(1140, 521, 1)
        self.conv2 = torch.nn.Conv1d(521, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 120, 1)

        self.bn1 = torch.nn.BatchNorm1d(521)
        self.bn2 = torch.nn.BatchNorm1d(256)

        self.drop = torch.nn.Dropout(0.2)
    def forward(self, inputs):
        x = inputs
        x = self.lstm(x)[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.drop(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.drop(x)
        x = self.conv3(x)
        return x.reshape(4,120)

In [63]:
net = Net()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0


[1,   200] loss: 1252260.297
[1,   400] loss: 1607374.644
[1,   600] loss: 1523639.462
[1,   800] loss: 2148098.919
[1,  1000] loss: 2100286.278
[1,  1200] loss: 1910400.954
[1,  1400] loss: 1861766.410
[1,  1600] loss: 1815994.726
[1,  1800] loss: 1673669.487
[1,  2000] loss: 1617192.509
[1,  2200] loss: 1545930.605
[1,  2400] loss: 1540218.615
[1,  2600] loss: 1441493.768
[1,  2800] loss: 1400581.814
[1,  3000] loss: 1334502.979
[1,  3200] loss: 1289691.495
[1,  3400] loss: 1209620.647
[1,  3600] loss: 1250113.216
[1,  3800] loss: 1204860.927
[1,  4000] loss: 1149831.371
[1,  4200] loss: 1066621.613
[1,  4400] loss: 1081109.162
[1,  4600] loss: 1073472.740
[1,  4800] loss: 969267.841


KeyboardInterrupt: 

In [23]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq, self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [ ]:
LSTM = LSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

In [105]:
class Encoder(nn.Module):

    def __init__(self, input_size, hidden_dim, num_layers=1):
        super(Encoder, self).__init__()

        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(self.input_size, self.hidden_dim, num_layers=self.num_layers)
        self.hidden = None

    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim))

    def forward(self, inputs):
        # Push through RNN layer (the ouput is irrelevant)
        _, self.hidden = self.lstm(inputs, self.hidden)
        return self.hidden

In [113]:
class Decoder(nn.Module):

    def __init__(self, hidden_dim, num_layers=1):
        super(Decoder, self).__init__()
        # input_size=1 since the output are single values
        self.lstm = nn.LSTM(1, hidden_dim, num_layers=num_layers)
        self.out = nn.Linear(hidden_dim, 1)

    def forward(self, outputs, hidden, criterion):
        batch_size, num_steps = outputs.shape
        # Create initial start value/token
        input = torch.tensor([[0.0]] * batch_size, dtype=torch.float)
        # Convert (batch_size, output_size) to (seq_len, batch_size, output_size)
        input = input.unsqueeze(0)

        loss = 0
        for i in range(num_steps):
            # Push current input through LSTM: (seq_len=1, batch_size, input_size=1)
            #print(hidden[0].size())
            output, hidden = self.lstm(input, hidden)
            #print('here')
            # Push the output of last step through linear layer; returns (batch_size, 1)
            output = self.out(output[-1])
            # Generate input for next step by adding seq_len dimension (see above)
            input = output.unsqueeze(0)
            # Compute loss between predicted value and true value
            loss += criterion(output, outputs[:, i])
        return loss

In [115]:
encoder = Encoder(4, 120)
decoder = Decoder(120)
criterion = nn.MSELoss()
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)
a = 0
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        if (len(data[0]) < 4): 
            break
        inputs,labels = data
        inputs = inputs.transpose(1,0)
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        #print(inputs.shape[1])
        encoder.hidden = encoder.init_hidden(inputs.shape[1])
        #print(encoder.hidden[0].size())
        # Do forward pass through encoder
        hidden = encoder(inputs)
        #print(hidden[0].size())
        #a = hidden
        # Do forward pass through decoder (decoder gets hidden state from encoder)
        loss = decoder(labels, hidden, criterion)
        # Backpropagation
        loss.backward()
        # Update parameters
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

[1,   200] loss: 279803462.320


KeyboardInterrupt: 

# End of Lehan Li

torch.Size([1, 4, 120])

In [69]:
if __name__ == '__main__':

    # 5 is the number of features of your data points

    # Create optimizers for encoder and decoder
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    # Some toy data: 2 sequences of length 10 with 5 features for each data point
    inputs = [
        [
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
        ],
        [
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
            [0.5, 0.2, 0.3, 0.4, 0.1],
        ]
    ]

    inputs = torch.tensor(np.array(inputs), dtype=torch.float)
    # Convert (batch_size, seq_len, input_size) to (seq_len, batch_size, input_size)
    inputs = inputs.transpose(1,0)
    print(inputs.size())

    # 2 sequences (to match the batch size) of length 6 (for the 6h into the future)
    outputs = [ [0.1, 0.2, 0.3, 0.1, 0.2, 0.3], [0.3, 0.2, 0.1, 0.3, 0.2, 0.1] ]
    outputs = torch.tensor(np.array(outputs), dtype=torch.float)

    print(outputs.size())
    #
    # Do one complete forward & backward pass
    #
    # Zero gradients of both optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # Reset hidden state of encoder for current batch
    encoder.hidden = encoder.init_hidden(inputs.shape[1])
    # Do forward pass through encoder
    hidden = encoder(inputs)
    # Do forward pass through decoder (decoder gets hidden state from encoder)
    loss = decoder(outputs, hidden, criterion)
    # Backpropagation
    loss.backward()
    # Update parameters
    encoder_optimizer.step()
    decoder_optimizer.step()
    print("Loss:", loss.item())

torch.Size([10, 2, 5])
torch.Size([2, 6])
Loss: 0.2605845332145691


### Visualize the batch of sequences

In [ ]:
import matplotlib.pyplot as plt
import random

agent_id = 0

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

        
for i_batch, sample_batch in enumerate(val_loader):
    inp, out = sample_batch
    """TODO:
      Deep learning model
      training routine
    """
    show_sample_batch(sample_batch, agent_id)
    break